In [1]:
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt 
from datetime import datetime
from pathlib import Path
import collections
import hashlib
import numbers
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import axes3d
%matplotlib inline
import os

from tensorflow.python.eager import context
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.framework import tensor_util
from tensorflow.python.layers import base as base_layer
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import partitioned_variables
from tensorflow.python.ops import random_ops
from tensorflow.python.ops import tensor_array_ops
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops import variables as tf_variables
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util import nest
from tensorflow.contrib.rnn.python.ops.core_rnn_cell import _Linear
from matplotlib.colors import LinearSegmentedColormap

In [2]:
# uplading mnist data 

old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [41]:
# Training Parameters
learning_rate = 0.001
training_steps = 5000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input


In [42]:


def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.BasicRNNCell(num_hidden)

    # Get lstm cell output
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[:,-1,:], weights['out']) + biases['out']

In [43]:
tf.reset_default_graph()
graph=tf.Graph()
with graph.as_default():
    # Define weights
    weights = {
        'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([num_classes]))
    }
    X = tf.placeholder("float", [None, timesteps, num_input])
    Y = tf.placeholder("float", [None, num_classes])
    logits = RNN(X, weights, biases)
    prediction = tf.nn.softmax(logits)
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    # Evaluate model (with test logits, for dropout to be disabled)
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
        # predictions 
        #prediction=tf.nn.softmax(logits)
    tf.summary.histogram('prediction',prediction+1e-8)
    tf.summary.histogram('logits',logits+1e-8)
    tf.summary.scalar('loss',loss_op)
    merged_summary_op=tf.summary.merge_all()
    # define loss 
        #loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y))
        # optimization loop 
        #tf.summary.scalar('loss',loss_op)
        #tf.summary.histogram('logits',logits)    
        #optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
        #gradients=optimizer.compute_gradients(loss_op)
        #capped_gvs = [(tf.clip_by_norm(grad, 1.), var) if not var.name.startswith("dense") else (grad, var) for grad, var in gradients]
        #for _, var in gradients:
        #    if var.name.startswith("dense"):
        #        print(var.name)   
        #train_op=optimizer.apply_gradients(capped_gvs)
        # initialize variables 
    merged_summary_op=tf.summary.merge_all()
        
        #saver=tf.train.Saver()
    

In [44]:
log_dir = "logs/irnn/irnn"
Path(log_dir).mkdir(exist_ok=True, parents=True)
filelist = [ f for f in os.listdir(log_dir) if f.endswith(".local") ]
for f in filelist:
    os.remove(os.path.join(log_dir, f))

In [45]:

# write graph into tensorboard 
tb_writer = tf.summary.FileWriter(log_dir,graph)
# run a training session 
with tf.Session(graph=graph) as sess:
    sess.run(init)
    for step in range(1,training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x=batch_x.reshape((batch_size,timesteps,num_input))
        # run optimizaer 
        sess.run(train_op,feed_dict={X:batch_x, Y:batch_y})
        loss_train, acc_train= sess.run([loss_op, accuracy],feed_dict={X:batch_x, Y:batch_y})
        merged_summary=sess.run(merged_summary_op,feed_dict={X:batch_x, Y:batch_y})
        tb_writer.add_summary(merged_summary, global_step=step)
        #tb_writer.flush()
        # show interim performance 
        if step % display_step==0 or step==1 : 
            # get batch loss and accuracy 
            print('Step: {}, Train Loss: {:.3f}, Train Acc: {:.3f}'.format(
            step + 1, loss_train, acc_train))
            # write summary 
            #tb_writer.add_summary(acc,global_step=step)
            #tb_writer.flush()
            # evaluate performance on test data 
            test_X=mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
            test_Y=mnist.test.labels[:test_len]

    print("Optimization Finished!")
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step: 2, Train Loss: 3.782, Train Acc: 0.227
Step: 201, Train Loss: 1.304, Train Acc: 0.594
Step: 401, Train Loss: 0.748, Train Acc: 0.750
Step: 601, Train Loss: 0.649, Train Acc: 0.781
Step: 801, Train Loss: 0.503, Train Acc: 0.852
Step: 1001, Train Loss: 0.674, Train Acc: 0.805
Step: 1201, Train Loss: 0.568, Train Acc: 0.867
Step: 1401, Train Loss: 0.487, Train Acc: 0.875
Step: 1601, Train Loss: 0.380, Train Acc: 0.875
Step: 1801, Train Loss: 0.461, Train Acc: 0.875
Step: 2001, Train Loss: 0.367, Train Acc: 0.891
Step: 2201, Train Loss: 0.385, Train Acc: 0.875
Step: 2401, Train Loss: 0.268, Train Acc: 0.906
Step: 2601, Train Loss: 0.446, Train Acc: 0.883
Step: 2801, Train Loss: 0.286, Train Acc: 0.914
Step: 3001, Train Loss: 0.410, Train Acc: 0.898
Step: 3201, Train Loss: 0.294, Train Acc: 0.891
Step: 3401, Train Loss: 0.285, Train Acc: 0.891
Step: 3601, Train Loss: 0.253, Train Acc: 0.906
Step: 3801, Train Loss: 0.265, Train Acc: 0.906
Step: 4001, Train Loss: 0.226, Train Acc: 0.914